In [1]:
import tensorflow as tf

In [2]:
import tensorflow_probability as tfp #확률 관련 라이브러리리mport tensorflow_probability #확률 관련 라이브러리

In [ ]:
!pip install tensorflow-probability==0.11.1 #아무 숫자를 넣으면 사용 할 수있는 낮은 버전으로 재설치 하는 테크닉닉

In [3]:
dir(tfp)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'bijectors',
 'debugging',
 'distributions',
 'experimental',
 'glm',
 'layers',
 'math',
 'mcmc',
 'monte_carlo',
 'optimizer',
 'python',
 'random',
 'stats',
 'sts',
 'substrates',
 'util',
 'vi']

텐서플로우 추가 모듈 설치
- tensorflow hub, datasets, probability , addons

In [5]:
(X_train,_),(X_test,_) = tf.keras.datasets.mnist.load_data()

In [ ]:
X_train = X_train.reshape(60000,28,28,1)/255
X_test = X_test.reshape(60000,28,28,1)/255

In [6]:
import numpy as np

In [8]:
x=np.array([1,2,3,4])
np.where(x>2,100,0) #조건에 따라 값 바꿈꿈

array([  0,   0, 100, 100])

In [ ]:
np.where(X_train>0.5,1.0,0.0) #

In [10]:
tf.data.Dataset.from_tensor_slices(X_train) #shuffle, batch, prefetch

<TensorSliceDataset shapes: (28, 28), types: tf.uint8>

In [ ]:
tf.keras.layers.Input       #tensor
tf.keras.layers.InputLayers #layer

In [ ]:
tf.keras.layers.Lambda #lambda 레이어tf.keras.layers.Lambda #lambda 레이어

In [11]:
#크기 확인할때 하단과 같이 확인크기 확인할때 하단과 같이 확인
m =  tf.keras.Sequential([
            tf.keras.layers.Conv2D(32, 3, strides=(2, 2), activation='relu',input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(64, 3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(  )])
m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
Total params: 18,816
Trainable params: 18,816
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tf.keras.layers.Conv2DTranspos # 컨볼루션 전 원본을 찾음(deconvolution) -> 이미지 크기 커짐짐

상속을 통해서 모델 구축

In [25]:
class CVAE(tf.keras.models.Model):
  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim

    #encoder
    self.encoder = tf.keras.Sequential([
            #tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(32, 3, strides=(2, 2), activation='relu',input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(64, 3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(  ),
            #tf.keras.layers.Dense(latent_dim + latent_dim), 
            tf.keras.layers.Dense(latent_dim *2), 

        ])
     
    #decoder
    self.decoder = tf.keras.Sequential(        [
            tf.keras.layers.Dense(7*7*32, activation='relu',input_shape=(latent_dim,)),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),  #3차원 형태 변경
            tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding='same',activation='relu'),
            tf.keras.layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='relu'),
            tf.keras.layers.Conv2DTranspose(1, 3, padding='same') #strides=1 생략 가능
        ]    )
  def call(self): #실행할때 
    pass
  def encoder_summary(self): #추가 기능
    print(self.encoder.summary())

In [22]:
t=CVAE(32)
t.summary() #아직 built 되지 않음 

ValueError: ignored

In [23]:
t.encoder.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                147520    
Total params: 166,336
Trainable params: 166,336
Non-trainable params: 0
_________________________________________________________________


In [24]:
t.decoder.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1568)              51744     
_________________________________________________________________
reshape_4 (Reshape)          (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_transpose_12 (Conv2DT (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 28, 28, 1)         289       
Total params: 88,993
Trainable params: 88,993
Non-trainable params: 0
_________________________________________________________________


In [26]:
a=CVAE(64)
a.encoder_summary() #추가 기능

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 13, 13, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 64)          18496     
_________________________________________________________________
flatten_3 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               295040    
Total params: 313,856
Trainable params: 313,856
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#원본
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

  def __init__(self, latent_dim):
    super(CVAE, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
            tf.keras.layers.Conv2D(
                filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(
                filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            # No activation
            tf.keras.layers.Dense(latent_dim + latent_dim),
        ]
    )

    self.decoder = tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
            tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
            tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
            tf.keras.layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            tf.keras.layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding='same',
                activation='relu'),
            # No activation
            tf.keras.layers.Conv2DTranspose(
                filters=1, kernel_size=3, strides=1, padding='same'),
        ]
    )
    #Sequential 단방향 모델

  @tf.function
  def sample(self, eps=None):
    if eps is None:
      eps = tf.random.normal(shape=(100, self.latent_dim))
    return self.decode(eps, apply_sigmoid=True)

  def encode(self, x): #평군, 분산으로 나눔
    mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1) 
    return mean, logvar

  def reparameterize(self, mean, logvar):
    eps = tf.random.normal(shape=mean.shape)
    return eps * tf.exp(logvar * .5) + mean

  def decode(self, z, apply_sigmoid=False):
    logits = self.decoder(z)
    if apply_sigmoid:
      probs = tf.sigmoid(logits)
      return probs
    return logits

[Dive into Deep Learning](http://d2l.ai/)
- 핵심 개념들이 구현되어 있음(기초)
- [Numerical Stability and Initialization](http://d2l.ai/chapter_multilayer-perceptrons/numerical-stability-and-init.html)
수치 안정성성

In [28]:
x=np.float32(1)
x

1.0

In [30]:
y=np.float32(1e-50)
y

0.0

In [33]:
1e3 == 1*10*10*10

True

In [35]:
z=x*y/y  #
z        #nan -> numerically unstable

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in float_scalars
  """Entry point for launching an IPython kernel.


nan

In [36]:
np.finfo(np.float32)  #최소,최대값, 타입에대한 정보보

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [38]:
np.nextafter(np.float(0)  ,np.float(1))

5e-324

In [ ]:
tf.keras.losses.BinaryCrossentropy(from_logits=True) #수치적 안정성 제공

In [39]:
def softmax(logits):
  exp = tf.exp(logits)
  return exp/tf.reduce_sum(exp)

In [40]:
softmax([1000. , 0.]).numpy()  #nan 으로 나옴 수치적 불안정성 

array([nan,  0.], dtype=float32)

incremental learning
- catastrophic forgetting: 기존에 학습 후 새로운 데이터를 학습하면 기존에 학습된 것을 까먹게 됨 
- 파라미터 초기화 중요


#### GradientTape
- 지원하지 않는 것을 새로 만들어야함

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)


def log_normal_pdf(sample, mean, logvar, raxis=1):
  log2pi = tf.math.log(2. * np.pi)
  return tf.reduce_sum(
      -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),
      axis=raxis)


def compute_loss(model, x):
  mean, logvar = model.encode(x) #encode 에서 split해서 구한 mean,logvar
  z = model.reparameterize(mean, logvar)
  x_logit = model.decode(z)      #decode 함수형 패러다임
  cross_ent = tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=x)
  logpx_z = -tf.reduce_sum(cross_ent, axis=[1, 2, 3])
  logpz = log_normal_pdf(z, 0., 0.)
  logqz_x = log_normal_pdf(z, mean, logvar)
  return -tf.reduce_mean(logpx_z + logpz - logqz_x)


@tf.function
def train_step(model, x, optimizer):
  """Executes one training step and returns the loss.

  This function computes the loss and gradients, and uses the latter to
  update the model's parameters.
  """
  with tf.GradientTape() as tape:
    loss = compute_loss(model, x)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# tensorflow 고급의 기본
1. layer
  - 제공하지 않는 모델 구현 tensorflow , pytorch
2. callbacks 
  - 학습 테크닉
3. tf.GradientTape
  - 학습도 조절
4. model

In [41]:
x=tf.Variable([1,2,3], dtype=tf.float32)

In [43]:
with tf.GradientTape()as t:
  y=x*x

In [44]:
t.gradient(y,x)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 6.], dtype=float32)>

In [ ]:
@tf.function()
@tf.function

#Tensorflow optimization

In [ ]:
tf.data.Dataset.prefetch  #최화

In [45]:
def x(a,b):
  return a+b

In [46]:
x(1,3)

4

In [47]:
x([1,2],[3,4])

[1, 2, 3, 4]

In [49]:
@np.vectorize
def x(a,b):
  return a+b

In [50]:
x([1,2],[3,4]) #넘파이 벡터 연산

array([4, 6])

### @tf.function
- 내부적으로 compiling
- 속도 최적화

In [51]:
@tf.function
def x(a,b):
  return a+b

In [52]:
x([1,2],[3,4]) #텐서로 바뀜

[<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>,
 <tf.Tensor: shape=(), dtype=int32, numpy=4>]

GAN은 모델은 쉬우나 학습테크닉을 배워야 함